<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/Sistem%20rekomendasi%20game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [124]:
df = pd.read_csv('/content/steam_games.csv', sep=",")
df

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

In [125]:
df = df[['name', 'release_date', 'game_description', 'popular_tags']]

In [126]:
df['release_year'] = pd.to_datetime(df['release_date'], format='mixed', errors='coerce').dt.year

<ipython-input-126-977f64f11d8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = pd.to_datetime(df['release_date'], format='mixed', errors='coerce').dt.year


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              40817 non-null  object 
 1   release_date      37654 non-null  object 
 2   game_description  37920 non-null  object 
 3   popular_tags      37888 non-null  object 
 4   release_year      36707 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.6+ MB


In [128]:
df = df[df['release_year'] >= 2010]

In [129]:
df['release_year'] = df['release_year'].astype(int)

<ipython-input-129-5746d9bf231b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_year'].astype(int)


In [130]:
df.shape

(35392, 5)

In [131]:
df.isnull().sum()/100

,0
name,0.00
release_date,0.00
game_description,0.51
popular_tags,1.07
release_year,0.00


In [132]:
df.dropna(inplace=True)

<ipython-input-132-c64f9f573c18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [133]:
df.shape

(35240, 5)

In [134]:
df['content'] = df['game_description'] + ' ' + df['popular_tags']

In [135]:
df.head()

,name,release_date,game_description,popular_tags,release_year,content
0,DOOM,"May 12, 2016","About This Game Developed by id software, the...","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...",2016,"About This Game Developed by id software, the..."
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Survival,Shooter,Multiplayer,Battle Royale,PvP...",2017,About This Game PLAYERUNKNOWN'S BATTLEGROUND...
2,BATTLETECH,"Apr 24, 2018",About This Game From original BATTLETECH/Mec...,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2018,About This Game From original BATTLETECH/Mec...
3,DayZ,"Dec 13, 2018",About This Game The post-soviet country of Ch...,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...",2018,About This Game The post-soviet country of Ch...
6,Devil May Cry 5,"Mar 7, 2019",About This Game The Devil you know returns in...,"Action,Hack and Slash,Great Soundtrack,Demons,...",2019,About This Game The Devil you know returns in...


In [136]:
df = df[['name', 'content']]

In [138]:
df.reset_index(drop=True)

,name,content
0,DOOM,"About This Game Developed by id software, the..."
1,PLAYERUNKNOWN'S BATTLEGROUNDS,About This Game PLAYERUNKNOWN'S BATTLEGROUND...
2,BATTLETECH,About This Game From original BATTLETECH/Mec...
3,DayZ,About This Game The post-soviet country of Ch...
4,Devil May Cry 5,About This Game The Devil you know returns in...
...,...,...
35235,Rocksmith® 2014 Edition – Remastered – Sabaton...,"About This Content Play ""Ghost Division"" by S..."
35236,Rocksmith® 2014 Edition – Remastered – Stone T...,"About This Content Play ""Trippin’ on a Hole i..."
35237,Fantasy Grounds - Quests of Doom 4: A Midnight...,About This Content Quests of Doom 4: A Midni...
35238,Mega Man X5 Sound Collection,About This Content Get equipped with the stun...
